In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

# Encode the input text
input_text = "Script Type"
inputs = tokenizer(input_text, return_tensors="pt")

# Generate translated tokens
outputs = model.generate(**inputs)

# Decode the output tokens to get translated text
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(translated_text)


/home/raghav/Documents/codes/projects/AI_ML/IIT/.venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/raghav/Documents/codes/projects/AI_ML/IIT/.venv/lib64/python3.13/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


स्क्रिप्ट क़िस्म


In [2]:
from datasets import load_dataset

dataset = load_dataset("cfilt/iitb-english-hindi", split="train")


In [3]:
dataset[0]["translation"]

{'en': 'Give your application an accessibility workout',
 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-hi")

def preprocess(batch):
    en_texts = [item["en"] for item in batch["translation"]]
    hi_texts = [item["hi"] for item in batch["translation"]]

    inputs = tokenizer(en_texts, truncation=True, padding="max_length", max_length=128)
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(hi_texts, truncation=True, padding="max_length", max_length=128)

    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, batch_size=128)

Map:   0%|          | 0/1659083 [00:00<?, ? examples/s]/home/raghav/Documents/codes/projects/AI_ML/IIT/.venv/lib64/python3.13/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1659083/1659083 [12:34<00:00, 2199.52 examples/s]


In [13]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./opus-hi-finetuned",
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2,
    num_train_epochs=3,
    logging_steps=500
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [14]:
trainer.train()

/home/raghav/Documents/codes/projects/AI_ML/IIT/.venv/lib64/python3.13/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/raghav/Documents/codes/projects/AI_ML/IIT/.venv/lib64/python3.13/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


KeyboardInterrupt: 

In [1]:
import torch 
torch.cuda.is_available()

False